# Install Packages

In [3]:
!pip install numpy_ext
!pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


# Imports

In [4]:
import boto3
import pandas as pd
import glob
import numpy as np
import pickle
import io
import os

from datetime import datetime, date, timedelta
from s3 import write_to_s3, read_from_s3, list_s3_contents, pickle_dump_to_s3

# Read from S3

In [7]:
training_test_df_monthly = read_from_s3("train_test/train_test_21_days.csv")
training_test_df_monthly.date = pd.to_datetime(training_test_df_monthly.date)

Successful S3 get_object response. Status - 200


In [ ]:
lime_samples = training_test_df_monthly.sample(100000)

In [12]:
x_vals = ['alpha001', 'alpha002', 'alpha003', 'alpha004', 'alpha005', 'alpha006', 'alpha007', 
          'alpha008', 'alpha009', 'alpha010', 'alpha011', 'alpha012', 'alpha013', 'alpha014', 
          'alpha015', 'alpha016', 'alpha017', 'alpha018', 'alpha019','alpha020','alpha021',
          'alpha022','alpha023','alpha024','alpha025','alpha026','alpha027','alpha028',
          'alpha029','alpha030','alpha032','alpha033','alpha034','alpha035','alpha036',
          'alpha037','alpha038','alpha040','alpha041','alpha042','alpha043','alpha044',
          'alpha045','alpha046','alpha047','alpha049','alpha050','alpha051','alpha052',
          'alpha053','alpha055','alpha060','alpha061','alpha062','alpha064','alpha065',
          'alpha068','alpha074','alpha075','alpha078','alpha081','alpha085','alpha086',
          'alpha094','alpha095','alpha099','alpha101','T10Y_minus_2Y','T10Y_minus_2Y_1M_Change',
          'Inflation_Breakeven_Level','Inflation_Breakeven_1M_Change','Real_GDP_Level','GDP_Growth_2Q',
          'PMI_Surprise_Mom','PMI_1Q_Forecast_Change','nyt_news_ChatGPT_sentiment',
          'nyt_news_finbert_positive','nyt_news_finbert_negative','nyt_news_finbert_neutral',
          'nyt_news_lmd_negative_words_count','nyt_news_lmd_positive_words_count',
          'nyt_news_lmd_uncertain_words_count','nyt_news_lmd_litigious_words_count',
          'nyt_news_lmd_strong_modal_words_count','nyt_news_lmd_weak_modal_words_count',
          'nyt_news_word_count','nyt_news_news_count','nyt_news_has_nyt_news','analyst_news_lmd_negative_words_count',
          'analyst_news_lmd_positive_words_count','analyst_news_lmd_uncertain_words_count',
          'analyst_news_lmd_litigious_words_count','analyst_news_lmd_strong_modal_words_count',
          'analyst_news_lmd_weak_modal_words_count','analyst_news_finbert_positive',
          'analyst_news_finbert_negative','analyst_news_finbert_neutral','analyst_news_news_count',
          'analyst_news_has_analyst_news','us_news_lmd_negative_words_count',
          'us_news_lmd_positive_words_count','us_news_lmd_uncertain_words_count',
          'us_news_lmd_litigious_words_count','us_news_lmd_strong_modal_words_count',
          'us_news_lmd_weak_modal_words_count','us_news_finbert_positive','us_news_finbert_negative',
          'us_news_finbert_neutral','us_news_news_count','us_news_has_us_news']

# Read model

In [17]:
direction_model_lgbm = read_from_s3("model/LGBM.sav", format="pickle")[0]

Successful S3 get_object response. Status - 200


# Predict using model

In [19]:
all_preds_LGBM = direction_model_lgbm.predict(training_test_df_monthly[x_vals])

In [20]:
training_test_df_monthly["LGBMClassifier"] = all_preds_LGBM

# Output Model Interpretation

In [21]:
from lime.lime_tabular import LimeTabularExplainer

In [23]:
# creating the explainer function
explainer = LimeTabularExplainer(lime_samples[x_vals].values, mode="regression", feature_names=x_vals)

/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/sklearn/utils/extmath.py:986: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/sklearn/utils/extmath.py:991: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/sklearn/utils/extmath.py:1021: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


https://github.com/marcotcr/lime/blob/master/lime/explanation.py

In [ ]:
from tqdm.notebook import tqdm
import os

for ticker in tqdm(training_test_df_monthly.ticker.unique()):
    curr_ticker_vars = training_test_df_monthly[training_test_df_monthly.ticker == ticker].reset_index(drop=True)
    all_explanations = []
    
    curr_file = f"dashboard_data/LIME/{ticker}_LGBM_explanation.csv"
    for i in tqdm(range(len(curr_ticker_vars))):
        explanation = explainer.explain_instance(curr_ticker_vars[x_vals].iloc[i].values, direction_model_lgbm.predict)
        current_exp = pd.DataFrame(explanation.as_list())
        current_exp.columns = ["variable", "score_contribution"]
        current_exp["ticker"] = curr_ticker_vars.loc[i, "ticker"]
        current_exp["date"] = curr_ticker_vars.loc[i, "date"]
        current_exp["target"] = curr_ticker_vars.loc[i, "target"]
        current_exp["LGBM_Prediction"] = curr_ticker_vars.loc[i, "LGBMClassifier"]
        all_explanations.append(current_exp)
        
    all_explanations_df = pd.concat(all_explanations)
    write_to_s3(all_explanations_df.reset_index(drop=True), curr_file)
    all_explanations = []